<a href="https://colab.research.google.com/github/crisbpadilla/DataScience-M2M-course-Datatalent/blob/main/Capstonpjct1_cristobal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <b>Concrete manufacturing analysis</b>

  The data was obtain from kaggle (https://www.kaggle.com/datasets/vinayakshanawad/cement-manufacturing-concrete-dataset?resource=download). This data contains the raw materials in kg/m^3 used to manufacture the concrete mix, plus de values of their respective compreshive strenght (Mpa) developed during a specific period of time measured in days.

  For this project each compressive strenght (CS) will be classify accordingly:

  * Class 1 : CS <11 kg/m^3
  * Class 2 : 10 kg/m^3 < CS < 21 kg/m^3
  * Class 3 : 20 kg/m^3 < CS < 31 kg/m^3
  * Class 4 : 30 kg/m^3 < CS < 41 kg/m^3
  * Class 5 : CS> 40 kg/m^3

The data was normalized so for each column ( fly ash, coarse aggregate, fines aggregate,blast furnace slag) was calculate their relative frecuency to get the proportion or percentage in the mix. From it, the stack bar chart was obtain to observe the relative composition of the mix according to their compressive strenght.

Also, with this data a widget is created to interact with the days after manufacturing, so to determined the influence of the different materials in their CS.






In [68]:
import pandas as pd

file=pd.read_csv('concrete.csv')

file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cement        1030 non-null   float64
 1   slag          1030 non-null   float64
 2   ash           1030 non-null   float64
 3   water         1030 non-null   float64
 4   superplastic  1030 non-null   float64
 5   coarseagg     1030 non-null   float64
 6   fineagg       1030 non-null   float64
 7   age           1030 non-null   int64  
 8   strength      1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


The data was analysed and it showed that there is no missing values on the whole dataset.

In [69]:
file.head()
#compressive strenght is in MPa
# all the other variableas are in kg/m3
#age/days

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,141.3,212.0,0.0,203.5,0.0,971.8,748.5,28,29.89
1,168.9,42.2,124.3,158.3,10.8,1080.8,796.2,14,23.51
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,28,29.22
3,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28,45.85
4,154.8,183.4,0.0,193.3,9.1,1047.4,696.7,28,18.29


A new column named 'total' was added as the sum of the components of the concrete mix.

In [70]:
columns=['cement','slag','ash','water','superplastic','coarseagg','fineagg']
file['total']=file[columns].sum(axis=1)
file.head(3)

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength,total
0,141.3,212.0,0.0,203.5,0.0,971.8,748.5,28,29.89,2277.1
1,168.9,42.2,124.3,158.3,10.8,1080.8,796.2,14,23.51,2381.5
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,28,29.22,2356.7


In [71]:
for column in file.columns:

  if column not in ['age','strength']:
    file[column]=(file[column]/file['total'])

n_file=file.copy()


In [72]:
import numpy as np

def category(data):

  ranges = [np.arange(0.00, 10.00,0.01), np.arange(10.00, 20.00,0.01), np.arange(20.0, 30.0),np.arange(30.0, 40.0)]

  if 11 > data > 0 :
    return 'class_1'
  elif 21 > data > 10:
    return 'class_2'
  elif 31 > data > 20:
    return 'class_3'
  elif 41 > data > 30:
    return 'class_4'
  elif data > 40:
    return 'class_5'

n_file['Concrete_Class'] = n_file['strength'].apply(category)

n_file.head()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength,total,Concrete_Class
0,0.062053,0.093101,0.000000,0.089368,0.000000,0.426771,0.328708,28,29.89,1.0,class_3
1,0.070922,0.017720,0.052194,0.066471,0.004535,0.453832,0.334327,14,23.51,1.0,class_3
2,0.106081,0.000000,0.040608,0.079518,0.002334,0.406034,0.365426,28,29.22,1.0,class_3
3,0.120362,0.051584,0.000000,0.103167,0.000000,0.421719,0.303167,28,45.85,1.0,class_5
4,0.067755,0.080273,0.000000,0.084606,0.003983,0.458441,0.304942,28,18.29,1.0,class_2


In [73]:
# lo que busco es un dic={slag: con valores para cada clase, so 5 avg values of each col,etc}
cs_class=n_file.groupby('Concrete_Class')
categories=list(cs_class.groups.keys())
raw_mat=['cement','slag','ash','water','superplastic','coarseagg','fineagg']
colors=['#2a9293','#8d6708','#12100c','#1bb3e3','#879aa0','#d84816','#eae30b']
d_source={raw_mat[0]:[],raw_mat[1]:[],raw_mat[2]:[],raw_mat[3]:[],raw_mat[4] :[],raw_mat[5]:[],raw_mat[6]:[]
          }
for cat in cs_class.groups:
   data_g=cs_class.get_group(cat)

   d_source['cement'].append(data_g['cement'].mean())
   d_source['slag'].append(data_g['slag'].mean())
   d_source['ash'].append(data_g['ash'].mean())
   d_source['water'].append(data_g['water'].mean())
   d_source['superplastic'].append(data_g['superplastic'].mean())
   d_source['coarseagg'].append(data_g['coarseagg'].mean())
   d_source['fineagg'].append(data_g['fineagg'].mean())

d_source['category']=categories
d_source

{'cement': [0.07989379537731939,
  0.09861685255862837,
  0.10548704048805353,
  0.11882912445043314,
  0.1432000125975407],
 'slag': [0.04049034633375724,
  0.02026896287388607,
  0.03131650031876602,
  0.032847000712054354,
  0.03461459552579705],
 'ash': [0.01835994401065575,
  0.024936385487281894,
  0.027118373555408495,
  0.025530120229371563,
  0.01908081721384129],
 'water': [0.08388376497187613,
  0.07932532174082423,
  0.07941018144022749,
  0.07963203521455108,
  0.07374555311651555],
 'superplastic': [0.0009352208199911792,
  0.0017906579713299192,
  0.002117005402566729,
  0.0025881179851875596,
  0.003547100953689354],
 'coarseagg': [0.4206229021199133,
  0.4343146439635962,
  0.41945734833188175,
  0.412561447069496,
  0.4054355105446505],
 'fineagg': [0.3558140263664872,
  0.34074717540445326,
  0.335093550463096,
  0.3280121543389063,
  0.3203764100479656],
 'category': ['class_1', 'class_2', 'class_3', 'class_4', 'class_5']}

In [74]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_notebook
import random

output_notebook()

source=ColumnDataSource(data=d_source)
f=figure(title='concrete mix by class',x_range=categories,y_range=(0,1),height=500)
f.vbar_stack(raw_mat,x='category',width=0.6,color=colors,source=source,legend_label=raw_mat)
f.legend.orientation = "vertical"
f.add_layout(f.legend[0], 'right')
hover = HoverTool()
hover.tooltips = [('material','$name'),('value','@$name{0.000}')]

f.add_tools(hover)

show(f)


In [75]:
file.drop(columns='total').corr()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
cement,1.000000,-0.295548,-0.410003,-0.080366,0.038966,-0.269551,-0.354261,0.105727,0.479399
slag,-0.295548,1.000000,-0.310634,0.133036,0.029284,-0.265786,-0.269818,-0.038712,0.122056
ash,-0.410003,-0.310634,1.000000,-0.194064,0.396610,-0.005727,0.090062,-0.154927,-0.112445
water,-0.080366,0.133036,-0.194064,1.000000,-0.609457,0.056243,-0.218040,0.282325,-0.314379
superplastic,0.038966,0.029284,0.396610,-0.609457,1.000000,-0.424712,0.110391,-0.196237,0.353690
coarseagg,-0.269551,-0.265786,-0.005727,0.056243,-0.424712,1.000000,-0.269976,0.071732,-0.322508
fineagg,-0.354261,-0.269818,0.090062,-0.218040,0.110391,-0.269976,1.000000,-0.110832,-0.286873
age,0.105727,-0.038712,-0.154927,0.282325,-0.196237,0.071732,-0.110832,1.000000,0.328873
strength,0.479399,0.122056,-0.112445,-0.314379,0.353690,-0.322508,-0.286873,0.328873,1.000000


In [76]:
from bokeh.palettes import Category20
hover = HoverTool()
hover.tooltips = [('age','$name days'),('value','@$name samples')]
data_age=n_file.groupby('age')
dict_age={}
list_1=list(np.zeros(5))
list_2=list(np.zeros(5))
list_3=list(np.zeros(5))
list_4=list(np.zeros(5))
list_5=list(np.zeros(5))
list_6=list(np.zeros(5))
list_7=list(np.zeros(5))
list_8=list(np.zeros(5))
list_9=list(np.zeros(5))
list_10=list(np.zeros(5))
list_11=list(np.zeros(5))
list_12=list(np.zeros(5))
list_13=list(np.zeros(5))
list_14=list(np.zeros(5))
lista=[list_1,list_2,list_3,list_4,list_5,list_6,list_7,list_8,list_9,list_10,list_11,list_12,list_13,list_14]
color=[]
for l,age in enumerate(data_age.groups):
  a=data_age.get_group(age)['Concrete_Class'].value_counts()
  dict_a=a.to_dict()
  color.append(Category20[20][l])

  for key,value in dict_a.items():
    if key=='class_1':
      lista[l][0]=value
      dict_age[str(age)]=lista[l]
    if key=='class_2':
      lista[l][1]=value
      dict_age[str(age)]=lista[l]
    if key=='class_3':
      lista[l][2]=value
      dict_age[str(age)]=lista[l]
    if key=='class_4':
      lista[l][3]=value
      dict_age[str(age)]=lista[l]
    if key=='class_5':
      lista[l][4]=value
      dict_age[str(age)]=lista[l]

dict_age['category']=categories
print(dict_age)
age_elem=list(dict_age.keys())[0:14]

source_age=ColumnDataSource(data=dict_age)
b=figure(title='Samples per category according age', y_range=categories,height=500)

b.hbar_stack(age_elem,y='category',height=0.3,source=source_age,color=color,legend_label=age_elem)
# b.legend.orientation = "horizontal"
b.add_layout(b.legend[0], 'right')
b.xaxis.axis_label='number of samples'
b.add_tools(hover)
show(b)

{'1': [1, 1, 0.0, 0.0, 0.0], '3': [29, 53, 31, 18, 3], '7': [16, 44, 24, 18, 24], '14': [0.0, 10, 30, 16, 6], '28': [6, 47, 103, 134, 135], '56': [0.0, 0.0, 5, 20, 66], '90': [0.0, 0.0, 7, 23, 24], '91': [0.0, 0.0, 0.0, 0.0, 22], '100': [0.0, 0.0, 0.0, 14, 38], '120': [0.0, 0.0, 0.0, 3, 0.0], '180': [0.0, 0.0, 4, 9, 13], '270': [0.0, 0.0, 0.0, 2, 11], '360': [0.0, 0.0, 0.0, 2, 4], '365': [0.0, 0.0, 2, 3, 9], 'category': ['class_1', 'class_2', 'class_3', 'class_4', 'class_5']}


In [77]:
# n_file[(n_file['cement']<0.08) & (n_file['Concrete_Class']=='class_5')]

l=figure(title='age vs strenght',height=500)
l.scatter(x=n_file['age'],y=n_file['strength'])
l.xaxis.axis_label='age (days)'
l.yaxis.axis_label='strength (Mpa)'
show(l)

In [78]:
from bokeh.io import push_notebook
from ipywidgets import interact,SelectMultiple,IntRangeSlider,SelectionRangeSlider
from bokeh.layouts import gridplot

handle=show(gridplot([[l,b],[f,None]]),notebook_handle=True)

age=sorted(n_file['age'].unique())
strength=n_file['strength']
min_age=int(age[0])
max_age=int(age[-1])

select_age=SelectionRangeSlider(options=age,value=(min_age,max_age),description='Age range') #it allows me to select a in a range of specifics values determined by a list

@interact(ages=select_age)#This is a tupple value ex:(california,colorado)
def update(ages):
  val=list(ages)
  value=[]
  for a in age:
    if a in (range(int(val[0]),int(val[1]+1))):
      value.append(a)


  data= n_file[n_file['age'].isin(value)]

  cs_class=data.groupby('Concrete_Class')
  categories=list(cs_class.groups.keys())
  raw_mat=['cement','slag','ash','water','superplastic','coarseagg','fineagg']
  colors=['#2a9293','#8d6708','#12100c','#1bb3e3','#879aa0','#d84816','#eae30b']
  new_source={raw_mat[0]:[],raw_mat[1]:[],raw_mat[2]:[],raw_mat[3]:[],raw_mat[4] :[],raw_mat[5]:[],raw_mat[6]:[]
          }
  for cat in cs_class.groups:
    data_g=cs_class.get_group(cat)

    new_source['cement'].append(data_g['cement'].mean())
    new_source['slag'].append(data_g['slag'].mean())
    new_source['ash'].append(data_g['ash'].mean())
    new_source['water'].append(data_g['water'].mean())
    new_source['superplastic'].append(data_g['superplastic'].mean())
    new_source['coarseagg'].append(data_g['coarseagg'].mean())
    new_source['fineagg'].append(data_g['fineagg'].mean())

  new_source['category']=categories
  new_source
  source.data=new_source

  push_notebook(handle=handle)#rerender the plot with the changes
                                #that we had made


interactive(children=(SelectionRangeSlider(description='Age range', index=(0, 13), options=(1, 3, 7, 14, 28, 5…

In [85]:
s_class=n_file.groupby('strength')
categ=[str(a) for a in s_class.groups.keys()]

print(categ)
raw_mat=['cement','slag','ash','water','superplastic','coarseagg','fineagg']
colors=['#2a9293','#8d6708','#12100c','#1bb3e3','#879aa0','#d84816','#eae30b']
s_source={raw_mat[0]:[],raw_mat[1]:[],raw_mat[2]:[],raw_mat[3]:[],raw_mat[4] :[],raw_mat[5]:[],raw_mat[6]:[]
          }
for cat in s_class.groups:
   data_g=s_class.get_group(cat)

   s_source['cement'].append(data_g['cement'].mean())
   s_source['slag'].append(data_g['slag'].mean())
   s_source['ash'].append(data_g['ash'].mean())
   s_source['water'].append(data_g['water'].mean())
   s_source['superplastic'].append(data_g['superplastic'].mean())
   s_source['coarseagg'].append(data_g['coarseagg'].mean())
   s_source['fineagg'].append(data_g['fineagg'].mean())

s_source['category']=categ
s_source

st_source=ColumnDataSource(data=s_source)
g=figure(title='concrete mix by strength',x_range=categ,y_range=(0,1),height=500)
g.vbar_stack(raw_mat,x='category',width=0.6,color=colors,source=st_source,legend_label=raw_mat)
g.legend.orientation = "vertical"
g.add_layout(g.legend[0], 'right')
g.xaxis.axis_label_orientation=0.2
hover = HoverTool()
hover.tooltips = [('material','$name'),('vaue','@$name{0.000}')]

g.add_tools(hover)

show(g)

['2.33', '3.32', '4.57', '4.78', '4.83', '4.9', '6.27', '6.28', '6.47', '6.81', '6.88', '6.9', '6.94', '7.32', '7.4', '7.51', '7.68', '7.72', '7.75', '7.84', '8.0', '8.06', '8.2', '8.37', '8.49', '8.54', '9.01', '9.13', '9.31', '9.45', '9.56', '9.62', '9.69', '9.73', '9.74', '9.85', '9.87', '9.99', '10.03', '10.09', '10.22', '10.34', '10.35', '10.38', '10.39', '10.54', '10.73', '10.76', '10.79', '11.17', '11.36', '11.39', '11.41', '11.47', '11.48', '11.58', '11.65', '11.85', '11.96', '11.98', '12.05', '12.18', '12.25', '12.37', '12.45', '12.46', '12.47', '12.54', '12.55', '12.64', '12.73', '12.79', '12.84', '13.09', '13.12', '13.18', '13.2', '13.22', '13.29', '13.33', '13.36', '13.4', '13.46', '13.52', '13.54', '13.57', '13.62', '13.66', '13.71', '13.82', '14.14', '14.2', '14.31', '14.4', '14.5', '14.54', '14.59', '14.6', '14.64', '14.7', '14.8', '14.84', '14.94', '14.99', '15.03', '15.04', '15.05', '15.07', '15.09', '15.34', '15.36', '15.42', '15.44', '15.52', '15.53', '15.57', '15.58

In [84]:
from ipywidgets import Dropdown

handle=show(g,notebook_handle=True)

class_select=Dropdown(options=categories,value=categories[0],description='strength')

@interact(cat_sl=class_select)
def update(cat_sl):

  k=n_file[n_file['Concrete_Class']==cat_sl]
  news_class=k.groupby('strength')
  categ=[str(a) for a in news_class.groups.keys()]

  print(categ)
  raw_mat=['cement','slag','ash','water','superplastic','coarseagg','fineagg']
  colors=['#2a9293','#8d6708','#12100c','#1bb3e3','#879aa0','#d84816','#eae30b']
  new_s={raw_mat[0]:[],raw_mat[1]:[],raw_mat[2]:[],raw_mat[3]:[],raw_mat[4] :[],raw_mat[5]:[],raw_mat[6]:[]
          }
  for cat in s_class.groups:
    data_g=s_class.get_group(cat)

    new_s['cement'].append(data_g['cement'].mean())
    new_s['slag'].append(data_g['slag'].mean())
    new_s['ash'].append(data_g['ash'].mean())
    new_s['water'].append(data_g['water'].mean())
    new_s['superplastic'].append(data_g['superplastic'].mean())
    new_s['coarseagg'].append(data_g['coarseagg'].mean())
    new_s['fineagg'].append(data_g['fineagg'].mean())

  new_s['category']=categ
  new_s
  st_source.data=new_s
  g.x_range.factors=categ
  push_notebook(handle=handle)

interactive(children=(Dropdown(description='strength', options=('class_1', 'class_2', 'class_3', 'class_4', 'c…

In [81]:
print(g.x_range.factors)

['2.33', '3.32', '4.57', '4.78', '4.83', '4.9', '6.27', '6.28', '6.47', '6.81', '6.88', '6.9', '6.94', '7.32', '7.4', '7.51', '7.68', '7.72', '7.75', '7.84', '8.0', '8.06', '8.2', '8.37', '8.49', '8.54', '9.01', '9.13', '9.31', '9.45', '9.56', '9.62', '9.69', '9.73', '9.74', '9.85', '9.87', '9.99', '10.03', '10.09', '10.22', '10.34', '10.35', '10.38', '10.39', '10.54', '10.73', '10.76', '10.79', '11.17', '11.36', '11.39', '11.41', '11.47', '11.48', '11.58', '11.65', '11.85', '11.96', '11.98', '12.05', '12.18', '12.25', '12.37', '12.45', '12.46', '12.47', '12.54', '12.55', '12.64', '12.73', '12.79', '12.84', '13.09', '13.12', '13.18', '13.2', '13.22', '13.29', '13.33', '13.36', '13.4', '13.46', '13.52', '13.54', '13.57', '13.62', '13.66', '13.71', '13.82', '14.14', '14.2', '14.31', '14.4', '14.5', '14.54', '14.59', '14.6', '14.64', '14.7', '14.8', '14.84', '14.94', '14.99', '15.03', '15.04', '15.05', '15.07', '15.09', '15.34', '15.36', '15.42', '15.44', '15.52', '15.53', '15.57', '15.58